In [ ]:
pip install ctgan

In [ ]:
pip install scipy

In [ ]:
conda update --force-reinstall pandas

# imports

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from ctgan import CTGANSynthesizer
import gc
import numpy as np 
import pandas as pd
import itertools
import gc
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
from datetime import datetime
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import StratifiedKFold, GroupKFold
from sklearn.preprocessing import StandardScaler, QuantileTransformer, LabelEncoder, minmax_scale,MinMaxScaler,RobustScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.layers import Dense, Dropout, Input, InputLayer, Flatten, LayerNormalization, BatchNormalization


In [ ]:
gc.collect()

# Read Data

In [ ]:
pd.set_option("display.max_columns", None)

train_df = pd.read_csv("../input/tabular-playground-series-dec-2021/train.csv")
test_df = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv")
pseudolabels_df = pd.read_csv("../input/tps-dec-pseudolabels-v1/tps-dec-pseudolabels.csv")
sub_df = pd.read_csv("../input/tabular-playground-series-dec-2021/sample_submission.csv")

pseudolabels_df.head()

In [ ]:
print("The imbalanced class distribution:")
print((train_df.groupby('Cover_Type').Id.nunique() / len(train_df)).apply(lambda p: f"{p:.3%}"))
print("The imbalanced class number:")
print(train_df.groupby('Cover_Type').Id.nunique())

In [ ]:
train_df = train_df[train_df.Cover_Type != 5]
train_df = pd.concat([train_df, pseudolabels_df], axis=0)

In [ ]:
del pseudolabels_df
gc.collect()

In [ ]:
print("The imbalanced class distribution:")
print((train_df.groupby('Cover_Type').Id.nunique() / len(train_df)).apply(lambda p: f"{p:.3%}"))
print("The imbalanced class number:")
print(train_df.groupby('Cover_Type').Id.nunique())

In [ ]:
train_df.drop("Id", axis=1, inplace=True)
test_df.drop("Id", axis=1, inplace=True)

In [ ]:
train_df_4=train_df[train_df.Cover_Type==4]

In [ ]:
feature_cols = train_df.columns.tolist()
cnt_features = []
cat_features = []

for col in feature_cols:
    if train_df[col].dtype=='float64':
        cnt_features.append(col)
    else:
        cat_features.append(col)

## Generate data for cover_type == 4

In [ ]:
ctgan = CTGANSynthesizer(epochs=50)
ctgan.fit(train_df_4, cat_features)

In [ ]:
# Synthetic copy
samples = ctgan.sample(700)

concatenate dataframes

In [ ]:
frames = [train_df,samples]

train_df = pd.concat(frames)

In [ ]:
del samples, ctgan , cnt_features , cat_features , train_df_4 , frames
gc.collect()

In [ ]:
print("The imbalanced class number:")
print(train_df.groupby('Cover_Type').size())

In [ ]:
cols = ["Soil_Type7", "Soil_Type15"]

train_df.drop(cols, axis=1, inplace=True)
test_df.drop(cols, axis=1, inplace=True)

In [ ]:
idx = train_df[train_df["Cover_Type"] == 5].index
train_df.drop(idx, axis=0, inplace=True)

Renaming some columns with long names

In [ ]:
new_names = {
    "Horizontal_Distance_To_Hydrology": "x_dist_hydrlgy",
    "Vertical_Distance_To_Hydrology": "y_dist_hydrlgy",
    "Horizontal_Distance_To_Roadways": "x_dist_rdwys",
    "Horizontal_Distance_To_Fire_Points": "x_dist_firepts"
}

train_df.rename(new_names, axis=1, inplace=True)
test_df.rename(new_names, axis=1, inplace=True)

In [ ]:
le = LabelEncoder()
target = le.fit_transform(train_df["Cover_Type"])

# Feature Engineering

In [ ]:
train_df["Aspect"][train_df["Aspect"] < 0] += 360
train_df["Aspect"][train_df["Aspect"] > 359] -= 360

test_df["Aspect"][test_df["Aspect"] < 0] += 360
test_df["Aspect"][test_df["Aspect"] > 359] -= 360

In [ ]:
# Manhhattan distance to Hydrology
train_df["mnhttn_dist_hydrlgy"] = np.abs(train_df["x_dist_hydrlgy"]) + np.abs(train_df["y_dist_hydrlgy"])
test_df["mnhttn_dist_hydrlgy"] = np.abs(test_df["x_dist_hydrlgy"]) + np.abs(test_df["y_dist_hydrlgy"])

# Euclidean distance to Hydrology
train_df["ecldn_dist_hydrlgy"] = (train_df["x_dist_hydrlgy"]**2 + train_df["y_dist_hydrlgy"]**2)**0.5
test_df["ecldn_dist_hydrlgy"] = (test_df["x_dist_hydrlgy"]**2 + test_df["y_dist_hydrlgy"]**2)**0.5

In [ ]:
soil_features = [x for x in train_df.columns if x.startswith("Soil_Type")]
train_df["soil_type_count"] = train_df[soil_features].sum(axis=1)
test_df["soil_type_count"] = test_df[soil_features].sum(axis=1)

wilderness_features = [x for x in train_df.columns if x.startswith("Wilderness_Area")]
train_df["wilderness_area_count"] = train_df[wilderness_features].sum(axis=1)
test_df["wilderness_area_count"] = test_df[wilderness_features].sum(axis=1)

In [ ]:
features_Hillshade = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']

train_df["Hillshade_mean"] = train_df[features_Hillshade].mean(axis=1)
train_df['amp_Hillshade'] = train_df[features_Hillshade].max(axis=1) - train_df[features_Hillshade].min(axis=1) 

test_df["Hillshade_mean"] = test_df[features_Hillshade].mean(axis=1)
test_df['amp_Hillshade'] = test_df[features_Hillshade].max(axis=1) - test_df[features_Hillshade].min(axis=1) 

In [ ]:
train_df.loc[train_df["Hillshade_9am"] < 0, "Hillshade_9am"] = 0
test_df.loc[test_df["Hillshade_9am"] < 0, "Hillshade_9am"] = 0

train_df.loc[train_df["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0
test_df.loc[test_df["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0

train_df.loc[train_df["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0
test_df.loc[test_df["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0

train_df.loc[train_df["Hillshade_9am"] > 255, "Hillshade_9am"] = 255
test_df.loc[test_df["Hillshade_9am"] > 255, "Hillshade_9am"] = 255

train_df.loc[train_df["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255
test_df.loc[test_df["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255

train_df.loc[train_df["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255
test_df.loc[test_df["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255

In [ ]:
cols = [
    "Elevation",
    "Aspect",
    "mnhttn_dist_hydrlgy",
    "ecldn_dist_hydrlgy",
    "soil_type_count",
    "wilderness_area_count",
    "Slope",
    "x_dist_hydrlgy",
    "y_dist_hydrlgy",
    "x_dist_rdwys",
    "Hillshade_9am",
    "Hillshade_Noon",
    "Hillshade_3pm",
    "x_dist_firepts",
    "soil_type_count",
    "wilderness_area_count"
]

scaler = RobustScaler()
train_df[cols] = scaler.fit_transform(train_df[cols])
test_df[cols] = scaler.transform(test_df[cols])

In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
 
    return df

In [ ]:
train_df = reduce_mem_usage(train_df)
test_df = reduce_mem_usage(test_df)

In [ ]:
gc.collect()

# Modeling 

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
    tf_strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print("Running on TPU:", tpu.master())
except:
    tf_strategy = tf.distribute.get_strategy()
    print(f"Running on {tf_strategy.num_replicas_in_sync} replicas")
    print("Number of GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# Plot training history
def plot_history(history, *, n_epochs=None, plot_lr=False, plot_acc=True, title=None, bottom=None, top=None):
    """Plot (the last unique n_epochs epochs of) the training history"""
    plt.figure(figsize=(15, 6))
    from_epoch = 0 if n_epochs is None else len(history['loss']) - n_epochs
    
    # Plot training and validation losses
    plt.plot(np.arange(from_epoch, len(history['loss'])), history['loss'][from_epoch:], label='training loss')
    try:
        plt.plot(np.arange(from_epoch, len(history['loss'])), history['val_loss'][from_epoch:], label='Validation loss')
        best_epoch = np.argmin(np.array(history['val_loss']))
        best_val_loss = history['val_loss'][best_epoch]
        if best_epoch >= from_epoch:
            plt.scatter([best_epoch], [best_val_loss], c='r', label=f'Best val_loss = {best_val_loss:.5f}')
        if best_epoch > 0:
            almost_epoch = np.argmin(np.array(history['val_loss'])[:best_epoch])
            almost_val_loss = history['val_loss'][almost_epoch]
            if almost_epoch >= from_epoch:
                plt.scatter([almost_epoch], [almost_val_loss], c='orange', label='Second best val_loss')
    except KeyError:
        pass
    if bottom is not None: plt.ylim(bottom=bottom)
    if top is not None: plt.ylim(top=top)
    plt.gca().xaxis.set_major_locator(MaxNLocator(integer=True))
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='lower left')
    if title is not None: plt.title(title)
        
    # Plot validation metrics
    if plot_acc:
        best_epoch = np.argmax(np.array(history['val_acc']))
        best_val_acc = history['val_acc'][best_epoch]
        ax2 = plt.gca().twinx()
        ax2.plot(np.arange(from_epoch, len(history['loss'])), np.array(history['val_acc'][from_epoch:]), color='r', label='Validation accuracy')
        if best_epoch >= from_epoch:
            plt.scatter([best_epoch], [best_val_acc], c='r', label=f'Best val_acc = {best_val_acc:.5f}')
        ax2.set_ylabel('Accuracy')
        ax2.legend(loc='center right')
        
    # Plot learning rate
    if plot_lr:
        ax2 = plt.gca().twinx()
        ax2.plot(np.arange(from_epoch, len(history['loss'])), np.array(history['lr'][from_epoch:]), color='g', label='Learning rate')
        ax2.set_ylabel('Learning rate')
        ax2.legend(loc='upper right')
        
    plt.show()
    

In [ ]:
features = [f for f in test_df.columns if f != 'Id' and f != 'Cover_Type']

In [ ]:
%%time
EPOCHS = 200# increase the number of epochs if the training curve indicates that a better result is possible
VERBOSE = 0 # set to 0 for less output, or to 2 for more output
SINGLE_FOLD = False # set to True for a quick experiment and to False for full cross-validation
RUNS = 1 # should be 1. increase the number of runs only if you want see how the result depends on the random seed
BATCH_SIZE = 1024 # if you set this too high, the notebook will crash (out of memory)
FOLDS =20

def my_model(X):
    
    
    """Return a compiled Keras model"""
    model = Sequential()
    model.add(InputLayer(input_shape=(X.shape[-1])))

    model.add(Dense(512, activation='selu'))
    model.add(Dense(512, activation='selu'))
    model.add(Dropout(rate=0.1))
    model.add(BatchNormalization())
    
    model.add(Dense(256, activation='selu'))
    model.add(Dense(256, activation='selu'))
    model.add(BatchNormalization())
    
    model.add(Dense(128, activation='selu'))
    model.add(Dense(128, activation='selu'))
    model.add(BatchNormalization())
    
    model.add(Dense(64, activation='selu'))
    model.add(Dropout(rate=0.1))
    model.add(Dense(64, activation='selu'))
        
    # Add the final layer with the correct activation function
    # Adding kernel_regularizer=tf.keras.regularizers.l2(l2=0.03) didn't make a difference
    model.add(Dense(len(le.classes_), activation='softmax'))
    
    
    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), loss='sparse_categorical_crossentropy', metrics=['acc'])
    return model

# Make the results reproducible
np.random.seed(2021)
tf.random.set_seed(2021)

total_start_time = datetime.now()
score_list, test_pred_list1, history_list = [], [], []
oof_list = [np.full((len(train_df), len(le.classes_)), -1.0, dtype='float32') for run in range(RUNS)]
for run in range(RUNS):
    kf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=1)
    for fold, (train_idx, val_idx) in enumerate(kf.split(train_df, y=train_df.Cover_Type)):
        print(f"Fold {run}.{fold}")
        start_time = datetime.now()
        X_tr = train_df.iloc[train_idx]
        X_va = train_df.iloc[val_idx]
        y_tr = target[train_idx]
        y_va = target[val_idx]
        X_tr = X_tr[features]
        X_va = X_va[features]

        # train
        preproc = StandardScaler() 
        X_tr = preproc.fit_transform(X_tr)
        X_va = preproc.transform(X_va)
        
        model = my_model(X_tr)

        # Define two callbacks: ReduceLROnPlateau, EarlyStopping
        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, 
                               patience=5, verbose=VERBOSE)

        es = EarlyStopping(monitor="val_acc", patience=20, 
                           verbose=VERBOSE, mode="max", 
                           restore_best_weights=True)
        

        # train and save the model
        history = model.fit(X_tr, y_tr, 
                            validation_data=(X_va, y_va), 
                            epochs=EPOCHS,
                            verbose=VERBOSE,
                            batch_size=BATCH_SIZE, 
                            validation_batch_size=len(X_va),
                            shuffle=True,
                            callbacks=[lr, es])
        history_list.append(history.history)
        model.save(f"model1/model{run}.{fold}")
        
        # Inference for validation after last epoch of fold
        y_va_pred = model.predict(X_va, batch_size=len(X_va))
        oof_list[run][val_idx] = y_va_pred
        y_va_pred = np.argmax(y_va_pred, axis=1)

        # Evaluation
        accuracy = accuracy_score(y_va, y_va_pred)
        score_list.append((accuracy, datetime.now() - start_time))
        print(f"Fold {run}.{fold} | {str(datetime.now() - start_time)[-12:-7]} | Epochs: {len(history_list[-1]['loss'])} | Accuracy: {accuracy:.5f}")
        if run == 0: plot_history(history_list[-1], title=f"Accuracy: {accuracy:.5f}")

        # Inference for test: keep the predicted probabilities
        test_pred_list1.append(model.predict(preproc.transform(test_df[features]), batch_size=BATCH_SIZE))
        
        # Clean up the memory (it seems that Keras doesn't clean up everything at keyboard interrupts)
        del model, y_va_pred
        gc.collect()
        
        if SINGLE_FOLD: break

In [ ]:
# Create the submission file
test_df = pd.read_csv("../input/tabular-playground-series-dec-2021/test.csv")
sub = test_df[['Id']].copy()
sub['Cover_Type'] = le.inverse_transform(np.argmax(sum(test_pred_list1), axis=1)) # soft voting by adding the probabilities of all models in the ensemble
sub.to_csv('submission.csv', index=False)

# Plot the distribution of the test predictions
plt.figure(figsize=(10,3))
plt.hist(train_df['Cover_Type'], bins=np.linspace(0.5, 7.5, 8), density=True, label='train labels')
plt.hist(sub['Cover_Type'], bins=np.linspace(0.5, 7.5, 8), density=True, rwidth=0.7, label='Test predictions')
plt.xlabel('Cover_Type')
plt.ylabel('Frequency')
plt.gca().yaxis.set_major_formatter(PercentFormatter())
plt.legend()
plt.show()

sub.head()